# Reviews Company

In [2]:
# !conda install -c conda-forge sentence-transformers
# !pip install scikit-learn
# !pip install numpy pandas
# !pip show sentence-transformers

In [3]:
# import required packages
import pandas as pd
import numpy as np
import spacy
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, normalize
from spacy.matcher import PhraseMatcher
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [4]:
pd.set_option("display.max_colwidth",None)
# pd.reset_option("display.max_row",None)

In [5]:
data = pd.read_csv("Data_Merge_Reviews.csv")
data

,Job Title,Job Rating,Time,Job Status,Pros,Cons,Company_Name
0,Brand Ambassador,5.0,2025-08-08,"Former employee, more than 1 year","Flexible Hours, Supportive Team, Good Training, Fun Work Environment, Opportunity To Travel.","Inconsistent Hours, Limited Career Growth, Commission Structure Can Be Unclear, Standing For Long Hours, Sometimes Slow Communication.",Safaricom
1,Senior Mobile Developer,4.0,2025-07-28,"Current contractor, more than 1 year",Great For Gaining Experience On Large Scale Enterprise Applications,A Lot Of Shifting Priorities,Safaricom
2,Project Manager,4.0,2025-07-15,"Former employee, more than 3 years","Strong Support For Professional Growth Through Training Exposure To LargeScale, Impactful Projects Across Kenya Competitive Salary And PerformanceBased Bonuses",Occasional Tight Deadlines And HighPressure Timelines Bureaucracy Can Delay Project SignOffs,Safaricom
3,Ict Technician,5.0,2025-07-18,Former employee,The Working Experience Was Good,There Are No Major Drawbacks I Can Talk About,Safaricom
4,Intern,5.0,2025-07-15,"Former intern, less than 1 year",Great Work Environment. Regular Bonuses.,None I Can Think Of,Safaricom
...,...,...,...,...,...,...,...
25037,Planning officer,4.0,2022-03-31,Former employee,have good medical covers for employees,terms of employment are contract,Equity
25038,Senior business analyst,4.0,2022-05-14,Current employee,Has good Learning potential for staff,Working schedule is not flexible,Equity
25039,Senior program manager,3.0,2016-05-02,Former employee,Potential for great social impact,Can be slow at times,Equity
25040,Ban assurance officer,1.0,2022-10-17,"Former employee, more than 1 year",Medical insurance cover. Low interest loans for staff,Bad management Less than appropriate pay Extremely long work hours Extensive workload Toxic work culture,Equity


In [6]:
# using 
data2 = data

In [7]:
def split_info(df: pd.Series)-> pd.DataFrame:
    employment_type = []
    role_type = []
    duration = []
    for raw in df:
        data = raw.split()
        emp = rol = dur = "default"

        if len(data) >=2:
            emp = data[0]
            rol = data[1].replace(",", "")

        if "less than 1 year" in raw:
            dur = "<1 year"
        elif "more than 1 year" in raw:
            dur = "1 - 3 year"
        elif "more than 3 years" in raw:
            dur = ">3 year"
        employment_type.append(emp)
        role_type.append(rol)
        duration.append(dur)
         
    return pd.DataFrame({
       "Employment_Type":employment_type,
        "Role_Type":role_type,
        "Duration":duration
    })
new_cols = split_info(data2["Job Status"])

In [8]:
data2 = pd.concat([data2, new_cols], axis=1)
data2 = data2[
    ["Job Title", "Job Rating", "Time", "Job Status", 
     "Employment_Type", "Role_Type", "Duration",
     "Pros", "Cons", "Company_Name"]
]

In [9]:
# data2.drop(["Employment_Type", "Role_Type", "Duration"], axis=1, inplace=True)

In [10]:
data2

,Job Title,Job Rating,Time,Job Status,Employment_Type,Role_Type,Duration,Pros,Cons,Company_Name
0,Brand Ambassador,5.0,2025-08-08,"Former employee, more than 1 year",Former,employee,1 - 3 year,"Flexible Hours, Supportive Team, Good Training, Fun Work Environment, Opportunity To Travel.","Inconsistent Hours, Limited Career Growth, Commission Structure Can Be Unclear, Standing For Long Hours, Sometimes Slow Communication.",Safaricom
1,Senior Mobile Developer,4.0,2025-07-28,"Current contractor, more than 1 year",Current,contractor,1 - 3 year,Great For Gaining Experience On Large Scale Enterprise Applications,A Lot Of Shifting Priorities,Safaricom
2,Project Manager,4.0,2025-07-15,"Former employee, more than 3 years",Former,employee,>3 year,"Strong Support For Professional Growth Through Training Exposure To LargeScale, Impactful Projects Across Kenya Competitive Salary And PerformanceBased Bonuses",Occasional Tight Deadlines And HighPressure Timelines Bureaucracy Can Delay Project SignOffs,Safaricom
3,Ict Technician,5.0,2025-07-18,Former employee,Former,employee,default,The Working Experience Was Good,There Are No Major Drawbacks I Can Talk About,Safaricom
4,Intern,5.0,2025-07-15,"Former intern, less than 1 year",Former,intern,<1 year,Great Work Environment. Regular Bonuses.,None I Can Think Of,Safaricom
...,...,...,...,...,...,...,...,...,...,...
25037,Planning officer,4.0,2022-03-31,Former employee,Former,employee,default,have good medical covers for employees,terms of employment are contract,Equity
25038,Senior business analyst,4.0,2022-05-14,Current employee,Current,employee,default,Has good Learning potential for staff,Working schedule is not flexible,Equity
25039,Senior program manager,3.0,2016-05-02,Former employee,Former,employee,default,Potential for great social impact,Can be slow at times,Equity
25040,Ban assurance officer,1.0,2022-10-17,"Former employee, more than 1 year",Former,employee,1 - 3 year,Medical insurance cover. Low interest loans for staff,Bad management Less than appropriate pay Extremely long work hours Extensive workload Toxic work culture,Equity


In [11]:
data2["Employment_Type"].unique().tolist()

['Former', 'Current', 'KEY']

In [12]:
data2[data2["Employment_Type"] == "KEY"]

,Job Title,Job Rating,Time,Job Status,Employment_Type,Role_Type,Duration,Pros,Cons,Company_Name
467,Online Data Entry,5.0,2021-07-21,KEY NOT FOUND: jobLine.seasonal-current,KEY,NOT,default,The Company Pays Well And On Time,Commissions Were Very Few Also Allowances,Safaricom


In [13]:
data2.drop(467, axis=0, inplace=True)

In [14]:
data2.set_index("Job Title", inplace=True)

In [15]:
data2= data2.reset_index()

In [16]:
data2["Role_Type"].unique().tolist()

['employee', 'contractor', 'intern', 'temporary', 'freelancer']

In [17]:
# jobrole = {"intern":"Internship"}

In [18]:
data2["Role_Type"].replace("intern", "internship",inplace=True)

In [19]:
# Normalize text (lowercase, remove weird symbols, etc.).
data2["Pros"] = data2["Pros"].str.lower()

In [20]:
data2["Job Title"] = data2["Job Title"].str.lower()

In [21]:
# This command is used to change the it's to it is 
import contractions
data2["Cons"] = data2["Cons"].apply(lambda x: contractions.fix(x) if isinstance(x, str) else x)

In [22]:
data2[["Cons","Company_Name"]].head(50)

,Cons,Company_Name
0,"Inconsistent Hours, Limited Career Growth, Commission Structure Can Be Unclear, Standing For Long Hours, Sometimes Slow Communication.",Safaricom
1,A Lot Of Shifting Priorities,Safaricom
2,Occasional Tight Deadlines And HighPressure Timelines Bureaucracy Can Delay Project SignOffs,Safaricom
3,There Are No Major Drawbacks I Can Talk About,Safaricom
4,None I Can Think Of,Safaricom
5,Workload May Be Too Much Sometime,Safaricom
6,Tough Management Extreme Work Conditions Pressure Nepotism,Safaricom
7,Is Place That Offers Carrier Growth,Safaricom
8,No Growth Compensation For Customer Care Is Not Worth It,Safaricom
9,Have To Meet Strict Deadlines,Safaricom


In [23]:
from spellchecker import SpellChecker
spell = SpellChecker()

def correct_sentence(text):
    if not isinstance(text, str):
        text = ""
    words = text.split()
    correct_words = []

    for word in words:
        if word.lower() in spell:
            correct_words.append(word)
        else:
            correction = spell.correction(word)
            correct_words.append(correction if correction else word)
            
    return " ".join(correct_words)

In [24]:
# data2["Corrected_Cons"] = data2["Cons"].fillna("").apply(correct_sentence)
data2

,Job Title,Job Rating,Time,Job Status,Employment_Type,Role_Type,Duration,Pros,Cons,Company_Name
0,brand ambassador,5.0,2025-08-08,"Former employee, more than 1 year",Former,employee,1 - 3 year,"flexible hours, supportive team, good training, fun work environment, opportunity to travel.","Inconsistent Hours, Limited Career Growth, Commission Structure Can Be Unclear, Standing For Long Hours, Sometimes Slow Communication.",Safaricom
1,senior mobile developer,4.0,2025-07-28,"Current contractor, more than 1 year",Current,contractor,1 - 3 year,great for gaining experience on large scale enterprise applications,A Lot Of Shifting Priorities,Safaricom
2,project manager,4.0,2025-07-15,"Former employee, more than 3 years",Former,employee,>3 year,"strong support for professional growth through training exposure to largescale, impactful projects across kenya competitive salary and performancebased bonuses",Occasional Tight Deadlines And HighPressure Timelines Bureaucracy Can Delay Project SignOffs,Safaricom
3,ict technician,5.0,2025-07-18,Former employee,Former,employee,default,the working experience was good,There Are No Major Drawbacks I Can Talk About,Safaricom
4,intern,5.0,2025-07-15,"Former intern, less than 1 year",Former,internship,<1 year,great work environment. regular bonuses.,None I Can Think Of,Safaricom
...,...,...,...,...,...,...,...,...,...,...
25036,planning officer,4.0,2022-03-31,Former employee,Former,employee,default,have good medical covers for employees,terms of employment are contract,Equity
25037,senior business analyst,4.0,2022-05-14,Current employee,Current,employee,default,has good learning potential for staff,Working schedule is not flexible,Equity
25038,senior program manager,3.0,2016-05-02,Former employee,Former,employee,default,potential for great social impact,Can be slow at times,Equity
25039,ban assurance officer,1.0,2022-10-17,"Former employee, more than 1 year",Former,employee,1 - 3 year,medical insurance cover. low interest loans for staff,Bad management Less than appropriate pay Extremely long work hours Extensive workload Toxic work culture,Equity


In [25]:
# load spacy
nlp = spacy.load("en_core_web_lg")

# patterns = [
#            "ict technician", "ict officer", "ict manager", "ict consultant", "ict manager", "ict project manager",
#            "ict security analyst", "ict service desk analyst", "ict - software engineer", "ict applications support"]

# adding entity rule
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
unique_titles = data2["Job Title"].dropna().unique().tolist()
pattern_doc = [nlp.make_doc(title.lower()) for title in unique_titles]
matcher.add("JOB_ROLE", pattern_doc)

status_map = {
    "ft": "Full-time",
    "full-time": "Full-time",
    "full time": "Full-time",
    "intern": "Internship",
    "internship": "Internship",
    "intern (paid)": "Internship",
    "contract": "Contract",
    "contractor": "Contract"
}
mapping = {"one": 1.0, "two": 2.0,"three": 3.0, "four": 4.0,"five":5.0}

def clean_status(status):
    if pd.isna(status): return None
    if "intern" in status: return "Internship"
    if "contract" in status: return "Contract"
    if "full" in status: return "Full-time"
    status = status.lower().strip()
    return status_map.get(status, status.title())

# normalize the job rating
def clean_rating(rating):
    if pd.isna(rating): return None
    if isinstance(rating, str):
        rating = rating.lower().strip()
        if rating in rating_map:
            return rating_map(rating)
        match = re.search(r"[0-9.]+", rating)
        return float(match.group()) if match else None
    return float(rating)

# text cleaning with spacy
def spacy_clean(text):
    if not isinstance(text,str) or not text.strip():
        return "", []
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    matches = matcher(doc)
    job_roles = [doc[start:end].text for _, start, end in matches]
    return " ".join(tokens), {"NER": entities, "ROLES": job_roles}
        
def process_row(row):
    job_title_clean, job_title_entities = spacy_clean(row.get("Job Title", ""))
    pros_clean, pros_entities = spacy_clean(row.get("Pros", ""))
    cons_clean, cons_entities = spacy_clean(row.get("Cons", ""))

    return pd.Series({
        "Job Title Clean": job_title_clean,
        "Job Title Entities": job_title_entities,
        "Pros Clean": pros_clean,
        "Pros Entities": pros_entities,
        "Cons Clean": cons_clean,
        "Cons Entities": cons_entities,
        "Cleaned Status": clean_status(row.get("Job Status", "")),
        "Cleaned Rating": clean_rating(row.get("Job Rating", ""))
    })

data2_cleaned = data2.apply(process_row, axis=1)
print(data2_cleaned.head())

           Job Title Clean  \
0         brand ambassador   
1  senior mobile developer   
2          project manager   
3           ict technician   
4                   intern   

                                                                   Job Title Entities  \
0                   {'NER': [], 'ROLES': ['brand', 'brand ambassador', 'ambassador']}   
1  {'NER': [], 'ROLES': ['senior mobile developer', 'mobile developer', 'developer']}   
2                     {'NER': [], 'ROLES': ['project', 'project manager', 'manager']}   
3         {'NER': [('ict', 'ORG')], 'ROLES': ['ict', 'ict technician', 'technician']}   
4                                                    {'NER': [], 'ROLES': ['intern']}   

                                                                                                                          Pros Clean  \
0                                                flexible hour supportive team good training fun work environment opportunity travel   
1           

In [26]:
nlp("data scientist").similarity(nlp("machine learning engineer"))

0.5121792330607114

In [27]:
nlp("internship").similarity(nlp("full-time"))

0.22726951282021637

In [28]:
nlp("project manager").similarity(nlp("team lead"))

0.5455771796658736

In [29]:
data2.columns

Index(['Job Title', 'Job Rating', 'Time', 'Job Status', 'Employment_Type',
       'Role_Type', 'Duration', 'Pros', 'Cons', 'Company_Name'],
      dtype='object')

In [ ]:
# apply the cleaning
data2_cleaned["Employment_Type"] = data2_cleaned["Employment_Type"].apply(clean_status)

In [31]:
data2_cleaned["Job Rating"] = data2_cleaned["Job Rating"].apply(clean_rating)

In [32]:
data2_cleaned["Company_Name"] = data2_cleaned["Company_Name"].str.lower().str.replace('r[^a-zA-Z]','', regex=True).str.strip()

In [ ]:
data2_cleaned[["Job Title Entities", "Job Title"]].head(50)

In [ ]:
data2_cleaned

# Transformers implementation

In [37]:
data2_cleaned["Time"] = data2["Time"]
data2_cleaned["Employment_Type"] = data2["Employment_Type"]
data2_cleaned["Role_Type"] = data2["Role_Type"]
data2_cleaned["Duration"] = data2["Duration"]
data2_cleaned["Company_Name"] = data2["Company_Name"]

In [38]:
data2_cleaned['combined_text'] = (
    data2_cleaned['Job Title Clean'].fillna('') + ' ' +
    data2_cleaned['Pros Clean'].fillna('') + ' ' +
    data2_cleaned['Cons Clean'].fillna('') + ' ' +
    data2_cleaned['Role_Type'].fillna('')
)


In [ ]:
data2_cleaned

In [39]:
# load the transformer model
model = SentenceTransformer('all-MiniLM-L12-v2')

In [ ]:
# generating job Embeddings
job_embeddings = model.encode(data2_cleaned['combined_text'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/783 [00:00<?, ?it/s]

In [ ]:
# save Embeddings for later use
np.save("job_embeddings.npy", job_embeddings)

In [ ]:
# load embedding
job_embeddings = np.load("job_embeddings.npy")

In [ ]:
# handle user query
user_query = "I want a remote internship that pays well"

In [ ]:
# convert to embedding
user_embedding = model.encode([user_query])

In [ ]:
# compute similarity score
similarity_scores = cosine_similarity(user_embedding, job_embeddings)[0]
similarity_scores

In [ ]:
# rank and display top matches
data2_cleaned['similarity'] = similarity_scores
top_matches = data2_cleaned.sort_values(by='similarity', ascending=False).head(10)
top_matches[['Company_Name', 'Job Title Clean', 'similarity', 'Cleaned Rating', 'Role_Type']]

In [ ]:
data2_cleaned.head(1)

In [ ]:
# top_matches
# !pip install matplotlib seaborn scikit-learn wordcloud umap-learn

In [ ]:
# visualization of the model using t-sne clusters
tsne = TSNE(n_components=2, random_state=42, perplexity=30, learning_rate=200)
job_embeddings_2d = tsne.fit_transform(job_embeddings)
data2_cleaned['x'] = job_embeddings_2d[:, 0]
data2_cleaned['y'] = job_embeddings_2d[:, 1]

#plot
plt.figure(figsize=(18, 7))
plt.style.use('seaborn-v0_8-darkgrid')
plt.scatter(data2_cleaned['x'], data2_cleaned['y'], c=data2_cleaned['Cleaned Rating'], cmap='viridis', s=50, alpha=.7)
plt.colorbar(label="Job Ratings")
plt.title('Job Embedding Clusters (t-SNE)')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.show()

In [ ]:
# visualization using UMAP
import umap
reducer = umap.UMAP(n_neighbors=15, min_dist=.1, random_state=42)
job_embedding_umap = reducer.fit_transform(job_embeddings)

plt.figure(figsize=(18, 7))
plt.scatter(job_embedding_umap[:, 0],job_embedding_umap[:, 1],
           c=data2_cleaned['Cleaned Rating'], cmap='plasma', s=50, alpha=.7)
plt.colorbar(label="Job Ratings")
plt.title('Job Embedding Clusters (UMAP)')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.show()

In [ ]:
# word clouds for pros and cons of top matches
from wordcloud import WordCloud
pros_text = ' '.join(top_matches['Pros Clean'].dropna())
cons_text = ' '.join(top_matches['Cons Clean'].dropna())

# pros
plt.figure(figsize=(18, 7))
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(pros_text)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Top Pros Mentioned")
plt.show()

# cons
plt.figure(figsize=(18, 7))
wordcloud = WordCloud(width=800, height=400, background_color="red").generate(pros_text)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Cons Pros Mentioned")
plt.show()

In [ ]:
# Pick top 10 similar jobs and compare their text similarity
similar_jobs = top_matches['combined_text'].head(10).tolist()
sim_matrix = cosine_similarity(model.encode(similar_jobs))

plt.figure(figsize=(18,7))
sns.heatmap(sim_matrix, cmap='coolwarm', annot=True)
plt.title('Job Similarity Heatmap')
plt.show()

In [ ]:
# Normalize Job Ratings
scaler = MinMaxScaler()
data2_cleaned['Cleaned Rating'] = data2_cleaned['Cleaned Rating'].fillna(data2_cleaned['Cleaned Rating'].mean())
data2_cleaned['Rating_scaled'] = scaler.fit_transform(data2_cleaned[['Cleaned Rating']])

# structured Feature Engineering

In [ ]:
# Encode
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
status_encoded = encoder.fit_transform(data2_cleaned[['Role_Type']])

status_cols = encoder.get_feature_names_out(['Role_Type'])
status_df = pd.DataFrame(status_encoded, columns=status_cols)

data2_cleaned = pd.concat([data2_cleaned.reset_index(drop=True), status_df.reset_index(drop=True)], axis=1)

In [ ]:
# Combine Structured + Text Embeddings
structured_features = np.concatenate(
    [data2_cleaned[['Rating_scaled']].values, status_df.values],
    axis=1
)
# Combine with job embeddings
hybrid_features = np.concatenate([job_embeddings, structured_features], axis=1)

In [ ]:
# save for later
np.save("hybrid_features.npy", hybrid_features)

In [ ]:
hybrid_features

In [34]:
data2_cleaned.columns

Index(['Job Title Clean', 'Job Title Entities', 'Pros Clean', 'Pros Entities',
       'Cons Clean', 'Cons Entities', 'Cleaned Status', 'Cleaned Rating'],
      dtype='object')

In [ ]:
# normalize hybrid features
hybrid_features_norm = normalize(hybrid_features)